In [1]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
from datetime import datetime
from citipy import  citipy
import requests
from config import weather_api_key

In [2]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats, lngs)
coordinates = list(lat_lngs)

In [3]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
  city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
  
  # If the city is unique, then we will add it to the cities list.
  if city not in cities:
    cities.append(city)

In [4]:
# Use the print() function to display the latitude and longitude combinations.
city_df=pd.DataFrame(cities)
city_df

,0
0,busselton
1,pacific grove
2,nikolskoye
3,ribeira grande
4,port elizabeth
...,...
751,manoel urbano
752,kutum
753,bandar-e lengeh
754,rosetta


In [5]:
# Import the requests library.
import requests

# Import the API key.
from config import weather_api_key

In [6]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key
print(url)

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=4d8e135e0253fca38e1414147129a948


In [7]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1
# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        weather_description= city_weather['weather'][0]['description']
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                      "Country": city_country,
                      "Lat": city_lat,
                      "Lng": city_lng,
                      "Max Temp": city_max_temp,
                      "Humidity": city_humidity,
                      "Cloudiness": city_clouds,
                      "Wind Speed": city_wind,
                      "Current Description": weather_description})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")


Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | busselton
Processing Record 2 of Set 1 | pacific grove
Processing Record 3 of Set 1 | nikolskoye
Processing Record 4 of Set 1 | ribeira grande
Processing Record 5 of Set 1 | port elizabeth
Processing Record 6 of Set 1 | caravelas
Processing Record 7 of Set 1 | santa fe
Processing Record 8 of Set 1 | atuona
Processing Record 9 of Set 1 | sitka
Processing Record 10 of Set 1 | cape town
Processing Record 11 of Set 1 | loreto
Processing Record 12 of Set 1 | giresun
Processing Record 13 of Set 1 | jamestown
Processing Record 14 of Set 1 | butaritari
Processing Record 15 of Set 1 | bilma
Processing Record 16 of Set 1 | camana
Processing Record 17 of Set 1 | marienburg
Processing Record 18 of Set 1 | jurm
Processing Record 19 of Set 1 | roma
Processing Record 20 of Set 1 | porto seguro
Processing Record 21 of Set 1 | alghero
Processing Record 22 of Set 1 | tiksi
Processing Record 23 of Set 1 | hilo
Proc

In [8]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Busselton,AU,-33.6500,115.3333,60.15,71,79,23.02,light rain
1,Pacific Grove,US,36.6177,-121.9166,73.24,74,0,10.36,clear sky
2,Nikolskoye,RU,59.7035,30.7861,40.91,60,12,2.66,few clouds
3,Ribeira Grande,PT,38.5167,-28.7000,64.80,100,100,3.44,overcast clouds
4,Port Elizabeth,ZA,-33.9180,25.5701,53.89,87,0,3.44,clear sky
5,Caravelas,BR,-17.7125,-39.2481,70.41,77,6,5.70,clear sky
6,Santa Fe,AR,-31.6333,-60.7000,65.01,73,46,4.00,light rain
7,Atuona,PF,-9.8000,-139.0333,77.68,80,27,17.56,light rain
8,Sitka,US,57.0531,-135.3300,51.69,71,100,10.36,overcast clouds
9,Cape Town,ZA,-33.9258,18.4232,62.38,86,0,2.30,clear sky


In [9]:

# Create the output file (CSV).
output_data_file = "weather_data/WeatherPy_Database.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")